In [ ]:
#ESTABLISH ENVIRONMENT FIRST IN TERMINAL

# conda install --yes -c anaconda xarray
# conda install --yes -c conda-forge pynio
# conda install -c anaconda pandas

#REFERENCE: https://spire.com/tutorial/spire-weather-tutorial-intro-to-processing-grib2-data-with-python/
#DATA: https://mrms.ncep.noaa.gov/data/2D/
#ARCHIVE: https://mrms.agron.iastate.edu/

In [ ]:
#Import dependencies in environment
import xarray as xr
import pandas as pd

In [ ]:
#Read in GRIB2 data
filepath = './Downloads/...'
ds = xr.open_dataset(filepath, engine='pynio')
print(ds)

In [ ]:
#Establish GRIB2 data as variable for parsing
variable = ('VAR_209_6_36_P0_L102_GLL0')
dd = ds.get(variable)
print(dd)

In [ ]:
#Dataframe translation
df = dd.to_dataframe()
df

In [ ]:
#Extract latitudes and longitudes for mapping
latitudes = df.index.get_level_values('lat_0')
#latitudes
longitudes = df.index.get_level_values('lon_0')
#longitudes

In [ ]:
#Establish latitude and longitude range
map_function = lambda lon: (lon-360) if (lon>180) else lon
remapped_longitudes = longitudes.map(map_function)
df['longitude'] = remapped_longitudes
df['latitude'] = latitudes
df

In [ ]:
# RESERVED FOR FILTER
# min_lat = ''
# max_lat = ''
# min_lon = ''
# max_lon = ''
# lat_filter = (df['latitude'] >= min_lat) & (df['latitude' <= max_lat])
# lon_filter = (df['longitude'] >= min_lon) & (df['longitude' <= max_lon])
# df = df.loc[lat_filter & lon_filter]

In [ ]:
#Prep datetime attribute
from datetime import datetime, timedelta

def parse_datetime_from_filename(filename):
    parts = filename.split(".")
    date = parts[1]
    forecast_date = datetime.strptime(date, "%Y%m%d")
    issuance_time = parts[2]
    issuance_time = int(issuance_time[1:3])
    lead_time = parts[-2]
    lead_time = int(lead_time[1:])
    hours = issuance_time + lead_time
    forecast_time = forecast_date + timedelta(hours=hours)
    return str(forecast_time)
timestamp = parse_datetime_from_filename(filepath)
df["time"] = timestamp

In [ ]:
#Final Export to directory
df = df.loc[:, ["latitude", "longitude", "time", variable]]
df.to_csv("output_data.csv", index=False)